#Saeed

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
def read_file(file):
    with open(file,'r', encoding='utf-8') as file_content:
        return file_content.read()


import re
def find_regex(raw_text, regex_uri):
    return re.findall(regex_uri, raw_text)


def remove_regex(raw_text, regex_uris):
    temp_step = raw_text
    for uri in regex_uris:
        temp_step = re.sub(uri, "", temp_step)
    return temp_step


import pandas as pd
def te_parser(content, tweets):
    lines = content.split("\n")
    lines.pop(0) # column's names
    lines.pop(-1) # empty line
    for index, line in enumerate(lines):
        #print(index)
        tweets = tweets.append({"content": line.split(",")[-1].replace('"', '')}, ignore_index=True)
    return tweets


def cuhd_parser(content, tweets):
    lines = content.split('\n')
    lines.pop(-1) # empty line
    for index, line in enumerate(lines):
        #print(index)
        attributes = line.split("|")
        tweets = tweets.append({"content": attributes[2],
                                "retweets": int(attributes[5]),
                                "favs": int(attributes[6])}, ignore_index=True)
    return tweets

import nltk
nltk.download('punkt')
nltk.download('stopwords')
def normalizer(raw_text):
    words = nltk.word_tokenize(raw_text)
    tokens = [word for word in words if word.isalpha()] # remove punctuations
    tokens_minus_stop_words = [word for word in tokens if word not in nltk.corpus.stopwords.words()]
    stemmer = nltk.stem.PorterStemmer() 
    stemmed_tokens = [stemmer.stem(token) for token in tokens_minus_stop_words]
    return stemmed_tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Load Datasets

In [5]:
# You maybe need to change datasets['path'] to point to datasets on your system or VM
datasets = [{'title': 'TE', 'path': '/content/drive/MyDrive/hashtagify/text_emotion.txt', 'parser': te_parser},
            {'title': 'CUHD', 'path': '/content/drive/MyDrive/hashtagify/collected_user_history_data.txt', 'parser': cuhd_parser}]

In [6]:
tweets = pd.DataFrame({'content': pd.Series([], dtype='str'),
                       'retweets': pd.Series([], dtype='int'),
                       'favs': pd.Series([], dtype='int'),
                       'mentions': pd.Series([], dtype='object'), # 'dtype=list' is not available
                       'hashtags': pd.Series([], dtype='object')}) # same

In [9]:
for dataset in datasets:
    content = read_file(dataset['path'])
    tweets = dataset['parser'](content, tweets)
    print(f'{dataset["title"]} dataset parsed.')

TE dataset parsed.
CUHD dataset parsed.


In [10]:
tweets.favs = tweets.favs.fillna(int(tweets.favs.mean()))  
tweets.retweets = tweets.retweets.fillna(int(tweets.retweets.mean()))

In [11]:
mention_regex = '@.+? '
retweet_regex = 'RT'
hashtag_regex = r'#.+? '
url_regex = r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
tweets.mentions = tweets.content.apply(find_regex, args=(mention_regex,)) # export mentions
tweets.hashtags = tweets.content.apply(find_regex, args=(hashtag_regex,)) # export hashtags
# Delete mentions, hashtags, retweet notations from tweet content
tweets.content = tweets.content.apply(remove_regex, args=([mention_regex, retweet_regex, hashtag_regex, url_regex],)) 

In [13]:
tweets.content = tweets.content.apply(normalizer)

In [ ]:
tweets.to_csv("/content/drive/MyDrive/hashtagify/tweets.csv") # save preprocessed tweets into file in order to decrease computing time

In [14]:
# Load tweets.csv
import pandas as pd
tweets = pd.read_csv('/content/drive/MyDrive/hashtagify/tweets.csv', index_col='Unnamed: 0')
tweets

,content,retweets,favs,mentions,hashtags
0,"['know', 'listenin', 'bad', 'habit', 'earlier'...",2411.0,1.0,['@tiffanylue '],[]
1,"['layin', 'bed', 'headach', 'ughhhh', 'waitin'...",2411.0,1.0,[],[]
2,"['funer', 'ceremoni', 'gloomi', 'friday']",2411.0,1.0,[],[]
3,"['want', 'hang', 'friend', 'soon']",2411.0,1.0,[],[]
4,[],2411.0,1.0,[],[]
...,...,...,...,...,...
62885,"['you', 'know', 'els', 'thought', 'press', 'en...",217.0,0.0,['@JenLovesWords: '],"['#TrumpPressConference ', '#fact ']"
62886,"['trump', 'tweet', 'media', 'enemi', 'assum', ...",32631.0,0.0,['@StephenAtHome: '],[]
62887,"['defeat', 'wolf', 'you', 'stop', 'feed', 'air...",2.0,19.0,"['@noahmichelson ', '@billmaher ', '@huffpostq...",[]
62888,['awkward'],0.0,0.0,"['@mattdpearce ', '@vikkie ']",[]


In [15]:
tweets = tweets.drop('mentions', axis=1)

In [18]:
tweets['content_'] = tweets['content']
tweets['favs_'] = tweets['favs']
tweets['Retweets_'] = tweets['retweets']
tweets = tweets.drop('content', axis=1)
tweets = tweets.drop('favs', axis=1)
tweets = tweets.drop('retweets', axis=1)

In [27]:
tweets = tweets[tweets.hashtags.str.len() > 8] # 5 for ['#'] + 3 for small hashtags (maybe 7 will be a good choice too)

In [29]:
for column in tweets.columns.to_list():
    if column.__len__() < 3:
      tweets = tweets.drop(column, axis=1) 

In [ ]:
def extract_list(content):
    words = [token.replace('[', '').replace(']','').replace("'","").strip() for token in content.split(',')]
    return words

def extract_words(record):
    words = extract_list(record['content_'])
    for index, word in enumerate(words):
        if word in record.index.to_list():
            record[word] += 1
        else:
            record[word] = 1
    return record
tweets = tweets.apply(extract_words, axis=1)

In [ ]:
tweets = tweets.fillna(0)

In [ ]:
columns = [column for column in tweets.columns.to_list() if column not in ['content_', 'favs_', 'retweets_', 'hashtags']]
word_frequency = {}
for column in columns:
    word_frequency[column] = tweets[column].sum()

In [ ]:
# TF.IDF measures
def df(word, x):
    if word in word_frequency:
        return word_frequency[word]
    else:
        number_word_occurrences = x[x[word]!=0].shape[0]
        word_frequency[word] = number_word_occurrences 
        return number_word_occurrences
        

import numpy as np
def idf(word, x):
    number_emails = x.shape[0]
    return  np.log(number_emails/df(word, x))
    
    
def tf(word, training_email):
    return np.log(training_email[word] + 1)


def score_rf_idf(training_email, x, desicion_email_bag_of_words):
    score = 0
    intersection_words = [word for word in desicion_email_bag_of_words if word in x.columns]
    for word in intersection_words :
        score += tf(word, training_email) * idf(word, x)
    return score


def knn_predictor(similarity_vector, y, k):
    knn_indexes = similarity_vector.sort_values(key=lambda x: 1/x)[:k].index
    hashtags = []
    for knn_index in knn_indexes:
        hashtags.append(y.loc[knn_index])
    return hashtags


def predict_hashtags(raw_text: str, x, y, similarity_measure, k) -> bool:
    tokens = normalizer(raw_text)
    similarity_vector = x.apply(similarity_measure, axis=1, args=(x, tokens))
    hashtags = knn_predictor(similarity_vector, y, k)
    return hashtags


In [ ]:
output = predict_hashtags("Good Morning Everyone , Happy Nice Day :D",
             tweets.drop("content_", axis=1).drop("hashtags", axis=1), tweets['hashtags'],score_rf_idf, 5)

In [ ]:
hashtags = []
for hashtag_list in output:
    hashtags.extend(hashtag_list.split(","))
predicted_hashtags = [hashtag[2:][:-3] for hashtag in  hashtags]
predicted_hashtags = list(set(predicted_hashtags)) # unique hashtags

In [ ]:
tweets.to_csv('tweets.csv')

# Ali

In [ ]:
!pip install tweepy
import tweepy

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import datetime

## Set up developer Twitter authorities

In [ ]:
CONSUMER_KEY = 'uWgt6k1Mzl6IN1IiEKvHAHaNm'
CONSUMER_SECRET = 'PNsb7eZpt8AUdN0TzUNZOsvAaxajMmw094nvcTIVlRd2ne64yx'
ACCESS_KEY = '1061225034407206912-v5VBoAVQddfuZdSABSDZH8azyQXmLm'
ACCESS_SECRET = 'eiwbyRCFz1TeKXVFyIVAJ3RGqoJ5HLoMeqX4NaFs9lfrU'

In [ ]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)
api = tweepy.API(auth)

Provide the woeid codes of the regions from which you want to get the hashtags

In [ ]:
# Tehran
# Lucknow, Patna, Ranchi, Amritsar, Ahmedabad, Mumbai, Hyderabad, Bangalore, Chennai
# Chicago, Detroit, Los Angeles, NYC, Washington
# London, Manchester, Newcastle, Liverpool
# Paris
# Berlin

woeid = {"Worldwide": [1],
#        "Iran": [2251945],
         "India": [2295377, 2295381, 2295383, 2295388, 2295402, 2295411, 2295414, 2295420, 2295424],
         "US": [2379574, 2391585, 2442047, 2459115, 2514815],
         "UK": [44418, 28218, 30079, 26734],
         "France": [615702],
         "Germany": [638242]
        }

See all the trending hashtags on Twitter

In [ ]:
twitter_trends = []

for country in woeid:
    for state_code in woeid[country]:
        trends1 = api.trends_place(state_code)
        trends = [trend['name'][1:] for trend in trends1[0]['trends']]
        twitter_trends.extend(trends)
twitter_trends[:10], twitter_trends.__len__()

(['CePostaPerTe',
  'efensa y Justicia',
  'VamosColoColo',
  'respo',
  'anús',
  'ASMOM',
  'nufc',
  'porting',
  'AtatürkçüDeğilim',
  'azard'],
 1047)

Filter out the duplicate hashtags

In [ ]:
twitter_tags = []

for trend in twitter_trends:
    if trend not in twitter_tags:
        twitter_tags.append(trend)

print(len(twitter_tags), twitter_tags)

304 ['CePostaPerTe', 'efensa y Justicia', 'VamosColoColo', 'respo', 'anús', 'ASMOM', 'nufc', 'porting', 'AtatürkçüDeğilim', 'azard', 'raga', 'udamericana', 'ewcastle', 'arraguez', 'enzema', 'arry King', 'uazo', 'oates', 'achado de Assis', 'TREAM MISS YOU', 'hikamaru', 'arela', 'am nook', 'lavés', 'odric', 'heltenham', 'elipe Neto', 'oritiba', 'orales', 'IMENA 5M EN IG', 'cGregor', 'enny Henry', 'ازارد', 'ets', 'raian Romero', 'andanda', 'epe Sand', 'utin', 'auci Ouchie', 'nicks Go', 'omário', 'olari', 'aká', 'atkins', 'helvey', 'kungba', 'ülnaz Şırınga', 'ovetic', 'raziella', 'ouche', 'idharth Shukla In BB14', 'Release_Lalu_Yadav', 'MamtaBanerjeeKoJaiShriRam', 'roos', 'RahulVadiya', 'विधायक_हो_तो_सांगा_जैसा', 'ThankYouJimin', 'oden', 'asemiro', 'laves', 'endy', 'azquez', 'idane', 'realish', 'ालू यादव', 'OREVER WITH JIMIN', 'en stokes', 'ubina', 'ilan', 'minem', '्री राम', 'P2 IS COMING', 'alwa', 'sensio', 'hara', 'engal', 'saram', 'ean Winchester', 'nita Bose', 'idi', 'elcome ben', 'ay

## Instagram Hashtags
The Twitter hashtags are fetched! Now we will start with Instagram hashtags by searching and using webscraping on these websites to get trending Instagram hashtags.
*   [***All Hashtag***](https://www.all-hashtag.com/)
*   [***Best-Hashtags***](http://best-hashtags.com/)
*   [***Top-Hashtags***](https://top-hashtags.com/)

In [ ]:
def all_hashtag(keyword):
    try:
        url = "https://www.all-hashtag.com/library/contents/ajax_generator.php"
        payload = {'keyword': keyword,
                   'filter': 'top'}
        files = []
        headers = {}
        response = requests.request("POST", url, headers=headers, data=payload, files=files)
        text = str(response.text)
        soup = BeautifulSoup(text, features="html.parser")
        tags_str = ""
        for val in soup.find_all("div", class_="copy-hashtags"):
            tags_str = val.get_text()
        tags_str = tags_str.split(" ")
        tags = list()
        for tag in tags_str:
            if tag:
                tags.append(tag.replace("\n", ""))
        return tags
    except Exception as ex:
        print(ex)
        return False

In [ ]:
def best_hashtag_handler(keyword):
    try:
        url = "http://best-hashtags.com/hashtag/{}/".format(keyword)
        response = requests.request("POST", url)
        soup = BeautifulSoup(response.text, features="html.parser")
        hashtags = list()
        all_text = ''
        for val in soup.find_all("table", class_="table"):
            all_text = val.get_text()
        all_text = all_text.split("\n")
        for val in all_text:
            if val.startswith("#") and val != '#':
                hashtags.append(val)
        return hashtags
    except Exception as ex:
        print(ex)
        return False

In [ ]:
def top_hashtags(keyword):
    tags_with_popularity = {}
    try:
        pages = 4
        for i in range(1, pages + 1):
            url = 'https://top-hashtags.com/search/?q={}&opt=top&sp={}'.format(keyword, i)
            req = requests.get(url)
            html_doc = req.text
            soup = BeautifulSoup(html_doc, "html5lib")
            for litag in soup.find_all('li', {'class': 'i-row'}):
                currentKey = None
                for div in litag.find_all('div', {'class': 'i-tag'}):
                    currentKey = div.text
                for div in litag.find_all('div', {'class': 'i-total'}):
                    tags_with_popularity[currentKey] = div.text
        return tags_with_popularity
    except Exception as ex:
        print(ex)
        return False

## Text Preprocessing Tools

In [ ]:
import string
import nltk
from nltk.corpus import stopwords
nltk.download(['averaged_perceptron_tagger', 'stopwords', 'wordnet'])
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

stop_words = set(stopwords.words('english')) 

punctuation_to_remove = string.punctuation

### ***Emoji***s

In [ ]:
EMOTICONS = {
    u":‑\)":"Happy face or smiley",
    u":\)":"Happy face or smiley",
    u":-\]":"Happy face or smiley",
    u":\]":"Happy face or smiley",
    u":-3":"Happy face smiley",
    u":3":"Happy face smiley",
    u":->":"Happy face smiley",
    u":>":"Happy face smiley",
    u"8-\)":"Happy face smiley",
    u":o\)":"Happy face smiley",
    u":-\}":"Happy face smiley",
    u":\}":"Happy face smiley",
    u":-\)":"Happy face smiley",
    u":c\)":"Happy face smiley",
    u":\^\)":"Happy face smiley",
    u"=\]":"Happy face smiley",
    u"=\)":"Happy face smiley",
    u":‑D":"Laughing, big grin or laugh with glasses",
    u":D":"Laughing, big grin or laugh with glasses",
    u"8‑D":"Laughing, big grin or laugh with glasses",
    u"8D":"Laughing, big grin or laugh with glasses",
    u"X‑D":"Laughing, big grin or laugh with glasses",
    u"XD":"Laughing, big grin or laugh with glasses",
    u"=D":"Laughing, big grin or laugh with glasses",
    u"=3":"Laughing, big grin or laugh with glasses",
    u"B\^D":"Laughing, big grin or laugh with glasses",
    u":-\)\)":"Very happy",
    u":‑\(":"Frown, sad, andry or pouting",
    u":-\(":"Frown, sad, andry or pouting",
    u":\(":"Frown, sad, andry or pouting",
    u":‑c":"Frown, sad, andry or pouting",
    u":c":"Frown, sad, andry or pouting",
    u":‑<":"Frown, sad, andry or pouting",
    u":<":"Frown, sad, andry or pouting",
    u":‑\[":"Frown, sad, andry or pouting",
    u":\[":"Frown, sad, andry or pouting",
    u":-\|\|":"Frown, sad, andry or pouting",
    u">:\[":"Frown, sad, andry or pouting",
    u":\{":"Frown, sad, andry or pouting",
    u":@":"Frown, sad, andry or pouting",
    u">:\(":"Frown, sad, andry or pouting",
    u":'‑\(":"Crying",
    u":'\(":"Crying",
    u":'‑\)":"Tears of happiness",
    u":'\)":"Tears of happiness",
    u"D‑':":"Horror",
    u"D:<":"Disgust",
    u"D:":"Sadness",
    u"D8":"Great dismay",
    u"D;":"Great dismay",
    u"D=":"Great dismay",
    u"DX":"Great dismay",
    u":‑O":"Surprise",
    u":O":"Surprise",
    u":‑o":"Surprise",
    u":o":"Surprise",
    u":-0":"Shock",
    u"8‑0":"Yawn",
    u">:O":"Yawn",
    u":-\*":"Kiss",
    u":\*":"Kiss",
    u":X":"Kiss",
    u";‑\)":"Wink or smirk",
    u";\)":"Wink or smirk",
    u"\*-\)":"Wink or smirk",
    u"\*\)":"Wink or smirk",
    u";‑\]":"Wink or smirk",
    u";\]":"Wink or smirk",
    u";\^\)":"Wink or smirk",
    u":‑,":"Wink or smirk",
    u";D":"Wink or smirk",
    u":‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"X‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"XP":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"d:":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"=p":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u">:P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":-[.]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":S":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":‑\|":"Straight face",
    u":\|":"Straight face",
    u":$":"Embarrassed or blushing",
    u":‑x":"Sealed lips or wearing braces or tongue-tied",
    u":x":"Sealed lips or wearing braces or tongue-tied",
    u":‑#":"Sealed lips or wearing braces or tongue-tied",
    u":#":"Sealed lips or wearing braces or tongue-tied",
    u":‑&":"Sealed lips or wearing braces or tongue-tied",
    u":&":"Sealed lips or wearing braces or tongue-tied",
    u"O:‑\)":"Angel, saint or innocent",
    u"O:\)":"Angel, saint or innocent",
    u"0:‑3":"Angel, saint or innocent",
    u"0:3":"Angel, saint or innocent",
    u"0:‑\)":"Angel, saint or innocent",
    u"0:\)":"Angel, saint or innocent",
    u":‑b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"0;\^\)":"Angel, saint or innocent",
    u">:‑\)":"Evil or devilish",
    u">:\)":"Evil or devilish",
    u"\}:‑\)":"Evil or devilish",
    u"\}:\)":"Evil or devilish",
    u"3:‑\)":"Evil or devilish",
    u"3:\)":"Evil or devilish",
    u">;\)":"Evil or devilish",
    u"\|;‑\)":"Cool",
    u"\|‑O":"Bored",
    u":‑J":"Tongue-in-cheek",
    u"#‑\)":"Party all night",
    u"%‑\)":"Drunk or confused",
    u"%\)":"Drunk or confused",
    u":-###..":"Being sick",
    u":###..":"Being sick",
    u"<:‑\|":"Dump",
    u"\(>_<\)":"Troubled",
    u"\(>_<\)>":"Troubled",
    u"\(';'\)":"Baby",
    u"\(\^\^>``":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(\^_\^;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(~_~;\) \(・\.・;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-\)zzz":"Sleeping",
    u"\(\^_-\)":"Wink",
    u"\(\(\+_\+\)\)":"Confused",
    u"\(\+o\+\)":"Confused",
    u"\(o\|o\)":"Ultraman",
    u"\^_\^":"Joyful",
    u"\(\^_\^\)/":"Joyful",
    u"\(\^O\^\)／":"Joyful",
    u"\(\^o\^\)／":"Joyful",
    u"\(__\)":"Kowtow as a sign of respect, or dogeza for apology",
    u"_\(\._\.\)_":"Kowtow as a sign of respect, or dogeza for apology",
    u"<\(_ _\)>":"Kowtow as a sign of respect, or dogeza for apology",
    u"<m\(__\)m>":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(__\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(_ _\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"\('_'\)":"Sad or Crying",
    u"\(/_;\)":"Sad or Crying",
    u"\(T_T\) \(;_;\)":"Sad or Crying",
    u"\(;_;":"Sad of Crying",
    u"\(;_:\)":"Sad or Crying",
    u"\(;O;\)":"Sad or Crying",
    u"\(:_;\)":"Sad or Crying",
    u"\(ToT\)":"Sad or Crying",
    u";_;":"Sad or Crying",
    u";-;":"Sad or Crying",
    u";n;":"Sad or Crying",
    u";;":"Sad or Crying",
    u"Q\.Q":"Sad or Crying",
    u"T\.T":"Sad or Crying",
    u"QQ":"Sad or Crying",
    u"Q_Q":"Sad or Crying",
    u"\(-\.-\)":"Shame",
    u"\(-_-\)":"Shame",
    u"\(一一\)":"Shame",
    u"\(；一_一\)":"Shame",
    u"\(=_=\)":"Tired",
    u"\(=\^\·\^=\)":"cat",
    u"\(=\^\·\·\^=\)":"cat",
    u"=_\^=	":"cat",
    u"\(\.\.\)":"Looking down",
    u"\(\._\.\)":"Looking down",
    u"\^m\^":"Giggling with hand covering mouth",
    u"\(\・\・?":"Confusion",
    u"\(?_?\)":"Confusion",
    u">\^_\^<":"Normal Laugh",
    u"<\^!\^>":"Normal Laugh",
    u"\^/\^":"Normal Laugh",
    u"\（\*\^_\^\*）" :"Normal Laugh",
    u"\(\^<\^\) \(\^\.\^\)":"Normal Laugh",
    u"\(^\^\)":"Normal Laugh",
    u"\(\^\.\^\)":"Normal Laugh",
    u"\(\^_\^\.\)":"Normal Laugh",
    u"\(\^_\^\)":"Normal Laugh",
    u"\(\^\^\)":"Normal Laugh",
    u"\(\^J\^\)":"Normal Laugh",
    u"\(\*\^\.\^\*\)":"Normal Laugh",
    u"\(\^—\^\）":"Normal Laugh",
    u"\(#\^\.\^#\)":"Normal Laugh",
    u"\（\^—\^\）":"Waving",
    u"\(;_;\)/~~~":"Waving",
    u"\(\^\.\^\)/~~~":"Waving",
    u"\(-_-\)/~~~ \($\·\·\)/~~~":"Waving",
    u"\(T_T\)/~~~":"Waving",
    u"\(ToT\)/~~~":"Waving",
    u"\(\*\^0\^\*\)":"Excited",
    u"\(\*_\*\)":"Amazed",
    u"\(\*_\*;":"Amazed",
    u"\(\+_\+\) \(@_@\)":"Amazed",
    u"\(\*\^\^\)v":"Laughing,Cheerful",
    u"\(\^_\^\)v":"Laughing,Cheerful",
    u"\(\(d[-_-]b\)\)":"Headphones,Listening to music",
    u'\(-"-\)':"Worried",
    u"\(ーー;\)":"Worried",
    u"\(\^0_0\^\)":"Eyeglasses",
    u"\(\＾ｖ\＾\)":"Happy",
    u"\(\＾ｕ\＾\)":"Happy",
    u"\(\^\)o\(\^\)":"Happy",
    u"\(\^O\^\)":"Happy",
    u"\(\^o\^\)":"Happy",
    u"\)\^o\^\(":"Happy",
    u":O o_O":"Surprised",
    u"o_0":"Surprised",
    u"o\.O":"Surpised",
    u"\(o\.o\)":"Surprised",
    u"oO":"Surprised",
    u"\(\*￣m￣\)":"Dissatisfied",
    u"\(‘A`\)":"Snubbed or Deflated"
}
emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')

In [ ]:
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

### $Preprocess$ function

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = " ".join([stemmer.stem(word) for word in text.split()])
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])
    text = emoji_pattern.sub(r'', text)
    text = emoticon_pattern.sub(r'', text)
    return text

## User Input Text

In [ ]:
final.extend(trending_on_twitter[:5])

['#motivate',
 '#udamericana',
 '#followfriday',
 '#happ',
 '#patchestwtselfieday',
 '#postivevibe',
 '#SaturdayMorning',
 '#AtatürkçüDeğilim',
 '#iloveitwhen',
 '#Frida',
 '#success',
 '#azard',
 '#dbuc09',
 '#blesse',
 '#nofilter']

###Conslusion
Below cell is the result of it's below cells; we examined all of them and as we explain in proposal, we decied it's better just to use tweepy.

In [ ]:
# Sample Text
import time
tic = time.time()
text = "Good Morning Everyone , Happy Nice Day :D"
text = " ".join([word for word in str(text).split() if '@' not in word])
text = " ".join([word for word in str(text).split() if word not in stop_words])
text = text.translate(str.maketrans('', '', punctuation_to_remove))
twitter_df = pd.DataFrame({"hashtag": twitter_tags})
twitter_df['hashTagLower'] = twitter_df['hashtag'].apply(lambda x: x.lower())
trending_on_twitter = []

for word in text.split():
    word = word.lower()
    if word[0] == '#':
        word = word[1:]
    for i in range(twitter_df.shape[0]):
        if word in twitter_df.iloc[i]["hashTagLower"]:
            hashtag_to_add = '#' + twitter_df.iloc[i]["hashtag"]
            if hashtag_to_add not in trending_on_twitter:
                trending_on_twitter.append(hashtag_to_add)
            
print(f'Trending on twitter: {trending_on_twitter}')
tac = time.time()
print(tac-tic)

# The Final Output
predicted_hashtags.extend(trending_on_twitter[:5])

# The same approch with instagram hashtags...

# preprocess_text = preprocess_text(text)
# unique_text = []

# for txt in preprocess_text.split():
#     if txt not in unique_text:
#         unique_text.append(txt)
        
# unique_text = " ".join(unique_text)
# pos = nltk.pos_tag(unique_text.split()) 
# print(pos)
# tags = []
# filter_pos = ["VB", "VBN", "VBP", "VBZ", "RB", "RBS", "RBR", "JJ", "JJR", "JJS", "NN", "NNS"]

# for word in pos:
#     if word[1] in filter_pos:
#         tags.append(word[0])

# hashTags = []
# hashTagFor = []
# popularity = []
# common = []

# for tag in tags:
#     ht = []
#     ht.extend(all_hashtag(tag))
#     ht.extend(best_hashtag_handler(tag))
#     hashTags.extend(ht)
#     hashTagFor.extend([tag] * len(ht))
#     popularity.extend([0] * len(ht))
#     common.extend([1] * len(ht))

# for tag in tags:
#     tags_with_popularity = top_hashtags(tag)
#     for key in tags_with_popularity:
#         hashTags.append(key)
#         hashTagFor.append(tag) 
#         common.append(1)
#         val = tags_with_popularity[key]
#         #print(val)
#         if 'K' in val:
#             popularity.append(round(float(val[:-1]) * 1000, 2))
#         elif 'M' in val:
#             popularity.append(round(float(val[:-1]) * 1000000, 2))
#         else:
#              popularity.append(0)
# df = pd.DataFrame({'Word': hashTagFor, 'HashTag': hashTags, 'Popularity': popularity, 'Common': common})
# df['HashTagLower'] = df['HashTag'].apply(lambda x: x.lower()[1:])
# df['Popularity'] = pd.to_numeric(df['Popularity'], errors='coerce')
# df = df.drop_duplicates(subset='HashTag', keep='first')
# df = df.sort_values(by='Popularity', ascending=False)
# df.head()
# insta_hashtag_suggest = {'hashtag': [], 'score': [], 'popularity': []}

# count = 0
# targetCount = 26

# while count < targetCount and count < df.shape[0]:
#     score = 0
#     current_hashtag = df.iloc[count]['HashTag']
#     current_popularity = df.iloc[count]['Popularity']
#     if current_popularity >= 100000 and current_popularity <= 1000000:
#         score = 7
#     if current_popularity >= 1000000 and current_popularity <= 5000000:
#         score = 8
#     if current_popularity >= 5000000:  
#         score = 9
#     insta_hashtag_suggest['score'].append(score)
#     insta_hashtag_suggest['hashtag'].append(current_hashtag)
#     insta_hashtag_suggest['popularity'].append(current_popularity)
#     count += 1
# print('Trending hashtags on Instagram with popularity scores:')
# print('______________________________________________________\n')
# print(insta_hashtag_suggest)

# tac = time.time()
# print(tac-tic) 

Trending on twitter: ['#SaturdayMorning', '#patchestwtselfieday', '#AtatürkçüDeğilim', '#azard', '#udamericana', '#achado de Assis', '#odric', '#andanda', '#epe Sand', '#idharth Shukla In BB14', '#Release_Lalu_Yadav', '#RahulVadiya', '#oden', '#endy', '#idane', '#idi', '#KuttiStoryVideoSong', '#IOwnARidiculousAmountOf', '#alt Disney', '#andace', '#V Teddy', '#oldwire', '#ony DeAngelo', '#ared Butler', '#est Bloomfield', '#onovan Edwards', '#ord field', '#tafford', '#TrumpsNoteToBidenSaid', '#handler', '#SurvivingTheRoads', '#ewande', '#endrick', '#amie Redknapp', '#heridan Smith', '#adger', '#MaskedSingerUK', '#alverde', '#adonjic', '#alerdi', '#idibé', '#en Yedder', '#olland', '#DSDS', '#sportstudio', '#YesToNoCovid', '#Labbadia', '#arah Dingens', '#amstagabend', '#ahrheit oder Pflicht', '#andy Crush', '#ardai', '#inblick in sein Denken', '#andalf']
0.2682363986968994


In [ ]:
# Sample Text
text = "We want @love and support ❤️❤️❤️"

In [ ]:
text = " ".join([word for word in str(text).split() if '@' not in word])
text = " ".join([word for word in str(text).split() if word not in stop_words])
text = text.translate(str.maketrans('', '', punctuation_to_remove))
text

'We want support ❤️❤️❤️'

In [ ]:
twitter_df = pd.DataFrame({"hashtag": twitter_tags})
twitter_df['hashTagLower'] = twitter_df['hashtag'].apply(lambda x: x.lower())
twitter_df

,hashtag,hashTagLower
0,InaugurationDay,inaugurationday
1,ichelle,ichelle
2,MCA,mca
3,TrumpsLastDay,trumpslastday
4,hite House,hite house
...,...,...
249,iedersehen,iedersehen
250,aske,aske
251,erfügung,erfügung
252,ereidigung,ereidigung


Check if any hashtag is trending on the Twitter trending hashtag list we created earlier.

In [ ]:
trending_on_twitter = []

for word in text.split():
    word = word.lower()
    if word[0] == '#':
        word = word[1:]
    for i in range(twitter_df.shape[0]):
        if word in twitter_df.iloc[i]["hashTagLower"]:
            hashtag_to_add = '#' + twitter_df.iloc[i]["hashtag"]
            if hashtag_to_add not in trending_on_twitter:
                trending_on_twitter.append(hashtag_to_add)
            
print(f'Trending on twitter: {trending_on_twitter}')

Trending on twitter: ['#axwell', '#WednesdayWisdom']


Start pre-processing the text to get Instagram hashtags

In [ ]:
preprocess_text = preprocess_text(text)
unique_text = []

for txt in preprocess_text.split():
    if txt not in unique_text:
        unique_text.append(txt)
        
unique_text = " ".join(unique_text)
unique_text

'we want support'

In [ ]:
pos = nltk.pos_tag(unique_text.split()) 
print(pos)
tags = []
filter_pos = ["VB", "VBN", "VBP", "VBZ", "RB", "RBS", "RBR", "JJ", "JJR", "JJS", "NN", "NNS"]

for word in pos:
    if word[1] in filter_pos:
        tags.append(word[0])


[('we', 'PRP'), ('want', 'VBP'), ('support', 'NN')]


Search and extract all the hashtags from the above three websites, using the tags' list.

In [ ]:
hashTags = []
hashTagFor = []
popularity = []
common = []

In [ ]:
for tag in tags:
    ht = []
    ht.extend(all_hashtag(tag))
    ht.extend(best_hashtag_handler(tag))
    hashTags.extend(ht)
    hashTagFor.extend([tag] * len(ht))
    popularity.extend([0] * len(ht))
    common.extend([1] * len(ht))
print(hashTags)
print(hashTagFor)
print(popularity)
print(common)

['#want', '#Wanted', '#wantit', '#wantback', '#wantmore', '#wanttogoback', '#wants', '#WantToMove', '#wanttodie', '#wantone', '#wantagh', '#wantmylook', '#wantanr8', '#wantitmore', '#wanttobeskinny', '#wantthis', '#wantyou', '#wanting', '#wantagirlfriend', '#wantsummer', '#wantedbody', '#wanttosleep', '#wantitall', '#wantsummerback', '#wantsome', '#wanttoloseweight', '#wanttobefamous', '#wantitnow', '#wantittobetrue', '#wantrepreneur', '#wanted', '#wantit', '#wanttogoback', '#wantmore', '#wantback', '#wantone', '#wanttomove', '#wantyouback', '#wantogo', '#wantthis', '#wantsome', '#wanting', '#wantagh', '#wantyou', '#wanttogo', '#wantitmore', '#wanttobeskinny', '#wanttosleep', '#wantmoney', '#wanttogohome', '#support', '#supportsmallbusiness', '#supportlocal', '#supporthandmade', '#supportblackbusiness', '#supportourtroops', '#supportart', '#supportsmall', '#supportlocalbusiness', '#SupportLocalArtists', '#supportsmallbusinesses', '#supporter', '#supportlocalmusic', '#SUPPORTARTISTS', '

In [ ]:
for tag in tags:
    tags_with_popularity = top_hashtags(tag)
    for key in tags_with_popularity:
        hashTags.append(key)
        hashTagFor.append(tag) 
        common.append(1)
        val = tags_with_popularity[key]
        #print(val)
        if 'K' in val:
            popularity.append(round(float(val[:-1]) * 1000, 2))
        elif 'M' in val:
            popularity.append(round(float(val[:-1]) * 1000000, 2))
        else:
             popularity.append(0)
print(hashTags)
print(hashTagFor)
print(popularity)
print(common)

['#want', '#Wanted', '#wantit', '#wantback', '#wantmore', '#wanttogoback', '#wants', '#WantToMove', '#wanttodie', '#wantone', '#wantagh', '#wantmylook', '#wantanr8', '#wantitmore', '#wanttobeskinny', '#wantthis', '#wantyou', '#wanting', '#wantagirlfriend', '#wantsummer', '#wantedbody', '#wanttosleep', '#wantitall', '#wantsummerback', '#wantsome', '#wanttoloseweight', '#wanttobefamous', '#wantitnow', '#wantittobetrue', '#wantrepreneur', '#wanted', '#wantit', '#wanttogoback', '#wantmore', '#wantback', '#wantone', '#wanttomove', '#wantyouback', '#wantogo', '#wantthis', '#wantsome', '#wanting', '#wantagh', '#wantyou', '#wanttogo', '#wantitmore', '#wanttobeskinny', '#wanttosleep', '#wantmoney', '#wanttogohome', '#support', '#supportsmallbusiness', '#supportlocal', '#supporthandmade', '#supportblackbusiness', '#supportourtroops', '#supportart', '#supportsmall', '#supportlocalbusiness', '#SupportLocalArtists', '#supportsmallbusinesses', '#supporter', '#supportlocalmusic', '#SUPPORTARTISTS', '

Now the hashtags are extracted and we to create a DataFrame for further computations

In [ ]:
df = pd.DataFrame({'Word': hashTagFor, 'HashTag': hashTags, 'Popularity': popularity, 'Common': common})
df['HashTagLower'] = df['HashTag'].apply(lambda x: x.lower()[1:])
df['Popularity'] = pd.to_numeric(df['Popularity'], errors='coerce')
df = df.drop_duplicates(subset='HashTag', keep='first')
df = df.sort_values(by='Popularity', ascending=False)
df.head()

,Word,HashTag,Popularity,Common,HashTagLower
145,support,#stylistssupportingstylists,995600.0,1,stylistssupportingstylists
147,support,#wwsupport,929400.0,1,wwsupport
149,support,#swsupport,906000.0,1,swsupport
151,support,#womensupportwomen,878400.0,1,womensupportwomen
156,support,#lgbtsupport,782900.0,1,lgbtsupport


Convert $Common$ to Percent(%)


In [ ]:
df['Common'] = df['Common'] / 3
df

,Word,HashTag,Popularity,Common,HashTagLower
145,support,#stylistssupportingstylists,995600.0,0.333333,stylistssupportingstylists
147,support,#wwsupport,929400.0,0.333333,wwsupport
149,support,#swsupport,906000.0,0.333333,swsupport
151,support,#womensupportwomen,878400.0,0.333333,womensupportwomen
156,support,#lgbtsupport,782900.0,0.333333,lgbtsupport
...,...,...,...,...,...
49,want,#wanttogohome,0.0,0.333333,wanttogohome
48,want,#wantmoney,0.0,0.333333,wantmoney
44,want,#wanttogo,0.0,0.333333,wanttogo
38,want,#wantogo,0.0,0.333333,wantogo


**Setting up the Instagram hashtags with some customized scores based on popularity**

Take top 25 hashtags based on popularity

In [ ]:
insta_hashtag_suggest = {'hashtag': [], 'score': [], 'popularity': []}

count = 0
targetCount = 26

while count < targetCount and count < df.shape[0]:
    score = 0
    current_hashtag = df.iloc[count]['HashTag']
    current_popularity = df.iloc[count]['Popularity']
    if current_popularity >= 100000 and current_popularity <= 1000000:
        score = 7
    if current_popularity >= 1000000 and current_popularity <= 5000000:
        score = 8
    if current_popularity >= 5000000:  
        score = 9
    insta_hashtag_suggest['score'].append(score)
    insta_hashtag_suggest['hashtag'].append(current_hashtag)
    insta_hashtag_suggest['popularity'].append(current_popularity)
    count += 1

In [ ]:
print('Trending hashtags on Instagram with popularity scores:')
print('______________________________________________________\n')
print(insta_hashtag_suggest)
print(f'Trending hashtags on twitter: {trending_on_twitter}')

Trending hashtags on Instagram with popularity scores:
______________________________________________________

{'hashtag': ['#puppylove', '#selflove', '#iloveyou', '#foodlover', '#catlover', '#ilovemydog', '#inlove', '#naturelover', '#doglovers', '#makeuplover', '#catlovers', '#truelove', '#ilovemycat', '#photographylovers', '#skylovers', '#babylove', '#booklover', '#coffeelover', '#archilovers', '#ilovemyjob', '#fashionlover', '#architecturelovers', '#onelove', '#doglove', '#madewithlove', '#foodlovers'], 'score': [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9], 'popularity': [48590000.0, 41990000.0, 41050000.0, 40650000.0, 38660000.0, 38340000.0, 35950000.0, 26870000.0, 25350000.0, 23820000.0, 21410000.0, 21240000.0, 16950000.0, 15680000.0, 15290000.0, 15270000.0, 15240000.0, 15040000.0, 14660000.0, 14500000.0, 14350000.0, 14060000.0, 13880000.0, 12660000.0, 12250000.0, 11930000.0]}


In [ ]:
print(f'Trending hashtags on twitter: {trending_on_twitter}')

Trending hashtags on twitter: ['#UR SWEETHEART TAEHYUN', '#NidhiWentToHarvard', '#LaMatinaleWE', '#on WE', '#wegenmorgen', '#wegengestern', '#ATION LOVES RUBINA']


### Banned Hashtags
These are the banned hashtags:

In [ ]:
banned_tags = [
        'alone', 'always', 'armparty', 'adulting', 'assday', 'ass', 'abdl', 'assworship', 'addmysc', 'asiangirl',
        'beautyblogger', 'brain', 'boho', 'besties', 'bikinibody', 'costumes', 'curvygirls',
        'date', 'dating', 'desk', 'dm', 'direct', 'elevator', 'eggplant', 'edm', 
        'fuck', 'girlsonly', 'gloves', 'graffitiigers', 'happythanksgiving', 
        'hawks', 'hotweather', 'humpday', 'hustler', 'ilovemyinstagram', 'instababy', 'instasport', 'iphonegraphy', 'italiano', 'ice', 
        'killingit', 'kansas', 'kissing', 'kickoff', 'leaves', 'like', 'lulu', 'lean', 
        'master', 'milf', 'mileycyrus', 'models', 'mustfollow', 'nasty', 'newyearsday', 'nude', 'nudism', 'nudity', 
        'overnight', 'orderweedonline', 'parties', 'petite', 'pornfood', 'pushups', 'prettygirl', 
        'rate', 'ravens', 'samelove', 'selfharm', 'skateboarding', 'skype', 'snap', 'snapchat', 'single', 'singlelife', 'stranger',
        'saltwater', 'shower', 'shit', 'sopretty', 'sunbathing', 'streetphoto', 'swole', 'snowstorm', 'sun', 'sexy', 
        'tanlines', 'todayimwearing', 'teens', 'teen', 'thought', 'tag4like', 'tagsforlikes', 'thighs', 'undies', 'valentinesday',
        'workflow', 'wtf', 'xanax', 'youngmodel'
]